In [22]:
from typing import List
from cfgnet.network.network import NetworkConfiguration, Network
import random

repos = [
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/mall",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/apollo",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/litemall",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/piggymetrics",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/spring-boot-admin",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/mall-swarm",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/Spring-Cloud-Platform",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/pig",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/jetlinks-community",
    "/home/simisimon/GitHub/cval_evaluation/spring-projects/music-website"
]


In [23]:
def get_dependencies(repo_path: str) -> List:
    cfg = NetworkConfiguration(
        project_root_abs=repo_path,
        enable_internal_links=True,
        enable_static_blacklist=False,
        enable_all_conflicts=True
    )

    network = Network.init_network(cfg=cfg)

    links = list(network.links)

    return links


In [24]:
test_set = []

for repo_path in repos:
    repo_name = repo_path.split("/")[-1]
    links = get_dependencies(repo_path=repo_path)
    print(f"Num links for {repo_name}: {len(links)}")
    random_sampled_links = random.choices(links, k=5)
    test_set += random_sampled_links

Num links for mall: 1435


Num links for apollo: 3584
Num links for litemall: 906


  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/spring-boot-admin/spring-boot-admin-samples/spring-boot-admin-sample-servlet/src/main/resources/application.yml", line 2, column 1
but found another document
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/spring-boot-admin/spring-boot-admin-samples/spring-boot-admin-sample-servlet/src/main/resources/application.yml", line 60, column 1"
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/spring-boot-admin/spring-boot-admin-samples/spring-boot-admin-sample-zookeeper/src/main/resources/application.yml", line 1, column 1
but found another document
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/spring-boot-admin/spring-boot-admin-samples/spring-boot-admin-sample-zookeeper/src/main/resources/application.yml", line 31, column 1"


Num links for piggymetrics: 2913


Num links for spring-boot-admin: 7612
Num links for mall-swarm: 1836


found character '@' that cannot start any token
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/pig/pig-auth/src/main/resources/application.yml", line 6, column 11"
found character '@' that cannot start any token
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/pig/pig-boot/src/main/resources/application.yml", line 8, column 11"
found character '@' that cannot start any token
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/pig/pig-gateway/src/main/resources/application.yml", line 6, column 11"
found character '@' that cannot start any token
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/pig/pig-upms/pig-upms-biz/src/main/resources/application.yml", line 6, column 11"
found character '@' that cannot start any token
  in "/home/simisimon/GitHub/cval_evaluation/spring-projects/pig/pig-visual/pig-codegen/src/main/resources/application.yml", line 6, column 11"
found character '@' that cannot start any token
  in "/home/simisimon/GitHub/cva

Num links for Spring-Cloud-Platform: 2248


Num links for pig: 6116
Num links for jetlinks-community: 9328
Num links for music-website: 277


In [25]:
link_data = []

for link in test_set:
    link_data.append({
        "link_str": str(link),
        "project": str(link).split("::::")[0],
        "option_name": link.node_a.get_options(),
        "option_value": link.node_a.name,
        "option_file": link.artifact_a.name,
        "option_type": link.node_a.config_type,
        "option_technology": link.artifact_a.concept_name,
        "dependent_option_name": link.node_b.get_options(),
        "dependent_option_value": link.node_b.name,
        "dependent_option_file": link.artifact_b.name,
        "dependent_option_type": link.node_b.config_type,
        "dependent_option_technology": link.artifact_b.concept_name,
    })


In [26]:
import pandas as pd

df = pd.DataFrame(link_data)
df.to_csv("dependencies_with_rules.csv", index=False)

In [ ]:
from llama_index.core import PromptTemplate

TASK_PROMPT = PromptTemplate(
    "Carefully verify whether configuration option {nameA} of type {typeA} with value {valueA} in {fileA} of technology {technologyA} depends on configuration option {nameB} of type {typeB} with value {valueB} in {fileB} of technology {technologyB} or vice versa." 
)

SYSTEM_PROMPT = PromptTemplate(
    "You are an expert in validating intra-technology and cross-technology configuration dependencies.\n" 
    "You will be presented with configuration options found in the software project '{project}'.\n" 
    "Your task is to determine whether the given configuration options actually depend on each other based on value-equality.\n\n"
    "{definition_str}"
)

TASK_PROMPT = PromptTemplate(
    "Carefully verify whether configuration option {nameA} of type {typeA} with value {valueA} in {fileA} of technology {technologyA} depends on configuration option {nameB} of type {typeB} with value {valueB} in {fileB} of technology {technologyB} or vice versa." 
)

FORMAT_STR = """Respond in a JSON format as shown below:
{{
  "rationale": string, // Provide a concise explanation of whether and why the configuration options depend on each other due to value-equality.
  "uncertainty": integer, // Rate your certainty of this dependency on a scale from 0 (completely uncertain) to 10 (absolutely certain).
  "isDependency": boolean // True if a dependency exists, or False otherwise.
}}"""


VALUE_EQUALITY_DEFINITION_STR = """A value equality dependency specifies that certain configuration options must have identical values in order to function correctly.
Inconsistencies in these configuration values can lead to dependency conflicts of varying severity.
However, configuration dependencies based on value equality carry the risk of being false positives, as configuration options whose values are equal do not necessarily have to depend on each other."""


USER_PROMPT = PromptTemplate(
    "{task_str}\n\n"
    "{format_str}"
)



In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv("./.env")



OpenAI(api_key=os.getenv("OPENAI_KEY"))
df = pd.read_csv("test_dependencies.csv")


for index, row in df.iterrows():
    print(row[0])
    dependency = transform_link(link_str=row[0])



mall::::mall-portal/pom.xml::::project::::dependencies::::dependency_spring-boot-starter-amqp::::groupId::::org.springframework.boot <-> mall::::pom.xml::::project::::dependencies::::dependency_spring-boot-starter-aop::::groupId::::org.springframework.boot;yes;
mall::::mall-security/pom.xml::::project::::dependencies::::dependency_spring-boot-starter-security::::groupId::::org.springframework.boot <-> mall::::pom.xml::::project::::dependencies::::dependency_spring-boot-configuration-processor::::groupId::::org.springframework.boot;yes;
mall::::mall-admin/src/main/resources/application-dev.yml::::spring.redis.timeout::::300ms <-> mall::::mall-portal/src/main/resources/application-dev.yml::::spring.redis.timeout::::300ms;yes;
mall::::mall-mbg/pom.xml::::project::::groupId::::com.macro.mall <-> mall::::mall-security/pom.xml::::project::::parent_mall::::groupId::::com.macro.mall;yes;
mall::::document/docker/docker-compose-app.yml::::services::::mall-portal::::volumes::::/etc/localtime:/etc

/tmp/ipykernel_4182/4001302433.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])


In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT.format()
        },
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)
